In [1]:
from pathlib import Path

root_results_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences"
# Define the directory to search
directory = Path(root_results_path)

# Use rglob to search recursively ('**/*.csv') 
# or glob for just the current folder ('*.csv')
csv_files = []
for path in list(directory.rglob('*.csv')):
    if "tests" in str(path) or len(str(path).split('/')[-1])<20:
        continue
    csv_files.append(path)
csv_files

[PosixPath('/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/resume_results_old.csv'),
 PosixPath('/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/CESoftDiceclDiceLoss_step3_1_DAsafe_epoch_Final/TTA/overlap_0.5/th_0.4/CESoftDiceclDiceLoss_step3_1_DAsafe_epoch_Final_TTA_overlap_0.5_th_0.4.csv'),
 PosixPath('/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/CESoftDiceclDiceLoss_step3_1_DAsafe_epoch_Final/TTA/overlap_0.5/th_0.5/CESoftDiceclDiceLoss_step3_1_DAsafe_epoch_Final_TTA_overlap_0.5_th_0.5.csv'),
 PosixPath('/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/CESoftDiceclDiceLoss_step3_1_DAsafe_epoch_Final/TTA/overlap_0.5/th_0.6/CESoftDiceclDiceLoss_step3_1_DAsafe_epoch_Final_TTA_overlap_0.5_th_0.6.csv'),
 PosixPath('/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/CESoftDiceclDiceLoss_step3_1_DAsafe_epoch_Final/no_TTA/overlap_0.5/th_0.4/CESoftDiceclDiceLoss_step3_1_DAsa

In [9]:
import pandas as pd
import glob
import os
from datetime import datetime

# Show all rows
pd.set_option('display.max_rows', None)

# Show all columns
pd.set_option('display.max_columns', None)
# Set options to show all rows and columns
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_colwidth', None) # Prevents text inside cells from being cut off


rows_list = []
cols_to_remove = ['tif_paths', 'id', 'pred_paths']

for file in csv_files:
    filename = os.path.basename(file)
    c_time = os.path.getctime(file)
    dt_c_time = datetime.fromtimestamp(c_time)
    df = pd.read_csv(file)
    
    if len(df) >= 80:
        mean_row = df.iloc[[79]].copy()
        clean_row = mean_row.drop(columns=cols_to_remove, errors='ignore')
        
        # --- PARSING FILENAME ---
        clean_row['Losses'] = filename.split('_DA')[0] if '_DA' in filename else  filename.split('_epoch')[0]
        clean_row['epoch'] = filename.split('epoch_')[-1].split('_')[0] 
        clean_row['Train DA'] = filename.split('_DA')[-1].split('_')[0] if '_DA' in filename else  filename.split('_epoch')[0][-4:]
        clean_row['tta_status'] = 'no_TTA' if 'no_TTA' in filename else 'TTA'
        
        if 'overlap_' in filename:
            overlap_val = filename.split('overlap_')[1].split('_')[0]
            clean_row['overlap'] = float(overlap_val)
            
        if 'th_' in filename:
            th_val = filename.split('th_')[1].replace('.csv', '')
            clean_row['threshold'] = float(th_val)
        clean_row['dt_c_time'] = dt_c_time
        rows_list.append(clean_row)

# 2. Create the new combined DataFrame
combined_df = pd.concat(rows_list, ignore_index=True)

# 3. REORDER COLUMNS: Define the leading order
# We list exactly what we want first, then add the rest of the columns
fixed_cols = ['Losses', 'epoch', 'Train DA', 'tta_status', 'overlap', 'threshold']
other_cols = [c for c in combined_df.columns if c not in fixed_cols]

combined_df = combined_df[fixed_cols + other_cols]

# 4. View the result
combined_df.sort_values(['image_score'], ascending=False)
#combined_df.sort_values(['Losses', 'epoch', 'tta_status', 'image_score'], ascending=False)

,Losses,epoch,Train DA,tta_status,overlap,threshold,image_score,topo_score,surface_dice,voi_score,voi_split,voi_merge,dt_c_time
113,BCE_Tversky,model,safe,TTA,0.5,0.4,0.538749,0.174875,0.829974,0.559414,1.194520,1.461276,2026-02-06 05:09:51.368912
114,BCE_Tversky,model,safe,TTA,0.5,0.5,0.535272,0.170764,0.832006,0.550974,1.313335,1.439372,2026-02-06 09:16:18.336620
16,BCE_DSC,80,safe,TTA,0.5,0.4,0.532745,0.160850,0.834857,0.549401,1.359566,1.420806,2026-02-07 08:27:07.714225
28,CESoftDiceclDiceLoss_step2,model,safe,TTA,0.5,0.4,0.532363,0.177727,0.805526,0.563172,1.054056,1.555795,2026-02-14 09:56:01.380807
101,Focal_Tversky,40,safe,TTA,0.5,0.4,0.532247,0.170707,0.806994,0.567392,1.181943,1.392581,2026-02-08 22:07:44.065517
107,Focal_Tversky,150,safe,TTA,0.5,0.4,0.531876,0.169530,0.811622,0.562713,1.143649,1.475659,2026-02-10 11:40:06.039175
90,Focal_Tversky,160,safe,TTA,0.5,0.5,0.531302,0.161398,0.830045,0.549619,1.308247,1.459065,2026-02-08 13:53:10.197170
89,Focal_Tversky,160,safe,TTA,0.5,0.4,0.530881,0.164590,0.809893,0.565832,1.090323,1.498478,2026-02-08 09:46:53.445137
29,CESoftDiceclDiceLoss_step2,model,safe,TTA,0.5,0.5,0.530851,0.169098,0.817402,0.554375,1.187012,1.521672,2026-02-14 10:37:21.469030
102,Focal_Tversky,40,safe,TTA,0.5,0.5,0.530634,0.160527,0.826306,0.552196,1.355738,1.392910,2026-02-09 02:13:40.495773


In [3]:

import pandas as pd

# Basic save
combined_df.to_csv('/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/resume_results_old.csv', index=False)

# check individual results

In [4]:
df_1 = pd.read_csv("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/Focal_Tversky_CLDICE_DAsafe_epoch_40/no_TTA/overlap_0.5/th_0.5/Focal_Tversky_CLDICE_DAsafe_epoch_40_no_TTA_overlap_0.5_th_0.5.csv")
df_1.sort_values("topo_score", ascending=False)

,tif_paths,id,pred_paths,image_score,topo_score,surface_dice,voi_score,voi_split,voi_merge
19,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,2961547523,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.609153,0.345385,0.885777,0.558617,1.185232,1.448550
76,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,2821927657,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.587089,0.298709,0.866888,0.554471,1.516604,1.161796
15,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,3694884789,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.590520,0.277785,0.855989,0.593110,0.673306,1.613456
55,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,3922476405,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.572199,0.245832,0.882407,0.541736,1.476135,1.343594
30,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,1845578058,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.593638,0.241246,0.911542,0.577784,1.161015,1.274822
7,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,4233602520,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.569788,0.235926,0.856010,0.569733,0.811347,1.706017
59,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,4015222329,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.598688,0.234370,0.946170,0.563477,0.750287,1.832028
33,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,3977024865,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.555490,0.222052,0.833413,0.563371,1.100056,1.483372
64,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,206065757,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.535642,0.221884,0.775824,0.564396,1.354948,1.217738
45,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,3527825464,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.543965,0.221273,0.860075,0.504447,1.527310,1.747248


In [5]:
df_2 = pd.read_csv(
    "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/Focal_Tversky_DAsafe_epoch_150/TTA/overlap_0.5/th_0.5/Focal_Tversky_DAsafe_epoch_150_TTA_overlap_0.5_th_0.5.csv")
df_2[df['id'] == "2961547523"]

,tif_paths,id,pred_paths,image_score,topo_score,surface_dice,voi_score,voi_split,voi_merge
19,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,2961547523,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.588261,0.228073,0.926005,0.55925,1.261165,1.365861


In [6]:
# Merge the two dataframes on the 'id' column
combined_df = df_1.merge(df_2[['id', 'voi_score']], on='id', suffixes=('_1', '_2'))

combined_df['diff'] = combined_df['voi_score_1'] - combined_df['voi_score_2']
combined_df = combined_df.sort_values('diff', ascending=False)
combined_df = combined_df[['id', 'voi_score_1', 'voi_score_2', 'diff']]
print(f"mean: {combined_df['diff'].mean()}")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(combined_df)



mean: 0.021120248376921207


,id,voi_score_1,voi_score_2,diff
54,1471460767,0.683634,0.492451,0.191183
11,1734818300,0.567745,0.429846,0.137899
22,4105398542,0.593891,0.462558,0.131333
21,3608009641,0.600676,0.475133,0.125543
43,3198627305,0.619629,0.496479,0.123149
12,803213133,0.571838,0.452051,0.119788
61,2708764018,0.635725,0.538326,0.097399
69,2203617984,0.608410,0.512486,0.095924
36,1332121747,0.569555,0.480814,0.088741
50,2116132949,0.639729,0.561340,0.078389


# Copy all csv to an independent folder

In [7]:
from pathlib import Path
import shutil

# Define directories
base_path = Path("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences")
dest_root = Path("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/csv_results")

# Find all .csv files
csv_files = list(base_path.rglob("*.csv"))

for file in csv_files:
    file_str = str(file)
    
    # Filtering logic
    if '/tests/' in file_str or file.name.startswith('th_'):
        continue
    
    # Create the new path using pathlib (cleaner than .replace)
    relative_path = file.relative_to(base_path)
    dest_path = dest_root / relative_path

    # Create the folder structure in the destination root
    dest_path.parent.mkdir(parents=True, exist_ok=True)
    
    # Copy the file
    shutil.copy2(file, dest_path)
    print(f"Copied: {dest_path}")

Copied: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/csv_results/resume_results_old.csv
Copied: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/csv_results/CESoftDiceclDiceLoss_step3_1_DAsafe_epoch_Final/TTA/overlap_0.5/th_0.4/CESoftDiceclDiceLoss_step3_1_DAsafe_epoch_Final_TTA_overlap_0.5_th_0.4.csv
Copied: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/csv_results/CESoftDiceclDiceLoss_step3_1_DAsafe_epoch_Final/TTA/overlap_0.5/th_0.5/CESoftDiceclDiceLoss_step3_1_DAsafe_epoch_Final_TTA_overlap_0.5_th_0.5.csv
Copied: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/csv_results/CESoftDiceclDiceLoss_step3_1_DAsafe_epoch_Final/TTA/overlap_0.5/th_0.6/CESoftDiceclDiceLoss_step3_1_DAsafe_epoch_Final_TTA_overlap_0.5_th_0.6.csv
Copied: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/csv_results/CESoftDiceclDiceLoss_step3_1_DAsafe_epoch_Final/no_TTA/overlap_0.5/th_0.4/CESoftDiceclDiceLoss_step3_1_DAsafe_epoch_Final_no_TTA_overl